# Correspondence-driven plane-based M3C2 (PBM3C2) with known segmentation

<p style="color:red;"> <a style="font-weight: bold">
WARNING:</a> The implementation of this method is experimental and under active development.
</p>

In this notebook, we are extending the [PB-M3C2 implementation](pbm3c2.ipynb) to work with segmentation information that is already present in the input data. This is useful if you are embedding the calculation into a larger workflow where a segmentation has already been produced.

In [ ]:
import py4dgeo
import pooch

In [ ]:
py4dgeo.set_interactive_backend("vtk")

We are reading the two input epochs from XYZ files which contain a total of four columns: X, Y and Z coordinates, as well a segment ID mapping each point to a segment. The `read_from_xyz` functionality allows us to read additional data columns through its `additional_dimensions` parameter. It is expecting a dictionary that maps the column index to a column name.

In [ ]:
# Set up pooch to download data from Zenodo
p = pooch.Pooch(base_url="doi:10.5281/zenodo.16751963/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()

try:
    # Download and extract the dataset
    p.fetch("usage_data.zip", processor=pooch.Unzip(members=["usage_data"]))
    
    # Define path to the extracted data
    data_path = p.path / "usage_data.zip.unzip" / "usage_data"
    print(f"Data path: {data_path}")
    
    # Read XYZ files from the extracted directory
    epoch0, epoch1 = py4dgeo.read_from_xyz(
        data_path / "plane_horizontal_t1_segmented.xyz", 
        data_path / "plane_horizontal_t2_segmented.xyz",
        additional_dimensions={3: "segment_id"},
        delimiter=",",
    )
    
except Exception as e:
    print(f"Failed to download or extract data: {e}")

Again, we instantiate the algorithm. Due to fundamental differences in the algorithm workflow, we are using a separated algorithm class for this use case:

In [ ]:
alg = py4dgeo.PBM3C2WithSegments()

Next, we will read the segmented point cloud, which is part of the input epochs, and reconstruct the required segments from it. As a result, we get the same information that we got from the `export_segments_for_labelling` method in the [base PB-M3C2 implementation](pbm3c2.ipynb). Again, we need to provide labelling and can choose to do so either interactively or with external tools. In contrast to `export_segments_for_labelling`, `reconstruct_post_segmentation_output` only writes one file - the full segmentation information file (which defaults to `extracted_segments.seg`):

In [ ]:
xyz_epoch0, xyz_epoch1, segments = alg.reconstruct_post_segmentation_output(
    epoch0=epoch0,
    epoch1=epoch1,
)

Having completed the labelling process, we read it back in and start the trainging procedure:

In [ ]:
alg.training(
    extracted_segments_file_name=str(data_path / "extracted_segments.seg"),
    extended_y_file_name="testdata-labelling2.csv",
)

In [ ]:
distances, uncertainties = alg.compute_distances(epoch0, epoch1)

*Note*: When comparing distance results between this notebook and the [base algorithm notebook](pbm3c2.ipynb), you might notice, that results do not necessarily agree even if the given segmentation information is exactly the same as the one computed in the base algorithm. This is due to the reconstruction process in this algorithm being forced to select the segment position (exported as the *core point*) from the segment points instead of reconstructing the correct position from the base algorithm.